In [1]:
import sys
sys.path.append('../../../build')
import numpy as np
import libry as ry

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



Setup Generation

In [2]:
C = ry.Config()

In [3]:
C.clear()
C.addFile('/home/jay/git/optimization-course/rai/test/Kin/kin/2D.g')
C.view()

In [4]:
hand = C.addFrame(name="hand", parent="gripper")
hand.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
hand.setRelativePosition([0,0,-.1])
hand.setColor([1,1,0])

In [5]:
f = C.frame("hand")
print("position:", f.getPosition())
print("orientation:", f.getQuaternion())

position: [ 6.33012702e-01 -5.96410162e-01  4.44089210e-17]
orientation: [ 0.61237244  0.35355339  0.61237244 -0.35355339]


In [27]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)
# initial conditions
q[0] = 3
q[1] = 1
C.setJointState(q)

joint names:  ['joint1', 'joint2']
joint state:  [4. 1.]


Let's place a goal and and obstacle

In [28]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.8,0,0])
ball.setColor([0,1,0])

In [29]:
Obs = C.addFrame(name="Obs")
Obs.setShape(ry.ST.sphere, [.3])
Obs.setPosition([0,1,0])
Obs.setColor([1,0,0])

Run KOMO to grab the green ball

In [30]:
X0 = C.getFrameState()
obj1 = "ball";
arm = "gripper";

komo = C.komo_path(1.,20, 10., True);
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq);
komo.addObjective([1.], ry.FS.distance, [arm, obj1], ry.OT.eq, [1e2]);
komo.addObjective([.9,1.], ry.FS.positionDiff, ["hand", obj1], ry.OT.sos, [1e0]);
komo.addObjective(times=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);

In [32]:
komo.optimize()
komo.getReport()

{'F_qItself/2-#4': {'order': 2.0, 'type': 'sos', 'sos': 0.1020314176683606},
 'F_AccumulatedCollisions/0-#15': {'order': 0.0, 'type': 'eq'},
 'F_qLimits2/0-#15': {'order': 0.0, 'type': 'ineq'},
 'F_PairCollision/0-gripper-ball': {'order': 0.0,
  'type': 'eq',
  'eq': 0.0006269932136107603},
 'F_PositionDiff/0-hand-ball': {'order': 0.0,
  'type': 'sos',
  'sos': 0.009911266419262705},
 'F_qItself/1-#4': {'order': 1.0, 'type': 'eq', 'eq': 3.9052014995988316e-05},
 'sos': 0.11194268408762331,
 'ineq': 0.0,
 'eq': 0.0006660452286067486,
 'f': 0.0}

In [33]:
komo.view(False,"mySolution")

0

In [36]:
komo.view_play(False, .3)

0